# Long-Running Memory Demo (Python) - UserInfoMemory Feature

This notebook demonstrates the **UserInfoMemory** feature - a long-running memory system that extracts and remembers user information (name, persona) across conversations.

## What You'll Learn
- How to enable/disable memory with the `enable_memory` flag
- How UserInfoMemory extracts name and persona from user messages
- How agents use remembered information to personalize responses
- How memory persists across messages within a session
- Difference between memory-enabled and memory-disabled conversations
- Best practices for using long-running memory

## Prerequisites
Make sure the backend API is running on `http://localhost:8000`

```bash
cd Backend/python
python -m uvicorn main:app --reload
```

## Memory Feature Overview
- **Memory OFF (default)**: Agent responds generically, doesn't extract or remember user info
- **Memory ON**: Agent extracts name/persona and personalizes future responses

## Setup - Install Required Packages

In [1]:
# Install required packages
%pip install requests python-dotenv -q

import requests
import json
from datetime import datetime
from typing import Dict, Any, Optional, List
import time

print("? Packages installed and modules imported successfully!")

Note: you may need to restart the kernel to use updated packages.
? Packages installed and modules imported successfully!



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Configure API Connection

In [2]:
# API Configuration
API_BASE_URL = "http://localhost:8000"
TIMEOUT = 120  # 2 minutes

# Create session for connection pooling
session = requests.Session()

print(f"? Connected to API: {API_BASE_URL}")
print(f"?? UserInfoMemory Demo\n")

? Connected to API: http://localhost:8000
?? UserInfoMemory Demo



## Scenario 1: Memory DISABLED (Default Behavior)

Without enabling memory, the agent responds generically and doesn't extract user information.

In [13]:
print("?? Scenario 1: Memory DISABLED")
print("="*50 + "\n")

# First message - introduce yourself
message_no_memory_1 = {
    "message": "Hi! My name is Alice and I'm a software engineer.",
    "agents": ["azure_openai_agent"],
    "enable_memory": False  # Memory disabled
}

print(f"[Turn 1] ?? User: {message_no_memory_1['message']}")
print(f"         Memory: {'ON ??' if message_no_memory_1['enable_memory'] else 'OFF ??'}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=message_no_memory_1,
    timeout=TIMEOUT
)
response.raise_for_status()
result_no_mem_1 = response.json()

print(f"?? Agent: {result_no_mem_1.get('content', '')}")

# For memory-off demonstration, send follow-up in a fresh session
message_no_memory_2 = {
    "message": "What's my name and what do I do?",
    "session_id": None,  # force new session so prior details are not reused
    "agents": ["azure_openai_agent"],
    "enable_memory": False
}

print(f"\n[Turn 2] ?? User: {message_no_memory_2['message']}")
print(f"         Memory: {'ON ??' if message_no_memory_2['enable_memory'] else 'OFF ??'}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=message_no_memory_2,
    timeout=TIMEOUT
)
response.raise_for_status()
result_no_mem_2 = response.json()

print(f"?? Agent: {result_no_mem_2.get('content', '')}")
print(f"\n?? Note: With memory disabled and a fresh session, the agent should not remember the name/persona.")

?? Scenario 1: Memory DISABLED

[Turn 1] ?? User: Hi! My name is Alice and I'm a software engineer.
         Memory: OFF ??

?? Agent: Hi Alice! It’s great to meet you. As a software engineer, do you have any particular languages, frameworks, or projects you’re currently working on? How can I assist you today?

[Turn 2] ?? User: What's my name and what do I do?
         Memory: OFF ??

?? Agent: I don’t have access to your personal information unless you share it with me during our conversation. If you’d like, you can tell me your name and what you do, and I’ll remember it for the current session to personalize my responses!

?? Note: With memory disabled and a fresh session, the agent should not remember the name/persona.


## Scenario 2: Memory ENABLED

With memory enabled, the agent extracts and remembers your name and persona.

In [14]:
print("\n\n?? Scenario 2: Memory ENABLED")
print("="*50 + "\n")

# First message - introduce yourself with memory enabled
message_with_memory_1 = {
    "message": "Hi! My name is Bob and I'm a data scientist.",
    "agents": ["azure_openai_agent"],
    "enable_memory": True  # Memory enabled!
}

print(f"[Turn 1] ?? User: {message_with_memory_1['message']}")
print(f"         Memory: {'ON ??' if message_with_memory_1['enable_memory'] else 'OFF ??'}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=message_with_memory_1,
    timeout=TIMEOUT
)
result_with_mem_1 = response.json()
session_with_memory = result_with_mem_1["session_id"]

print(f"?? Agent: {result_with_mem_1['content']}")
print(f"\n?? Session ID: {session_with_memory}")
print(f"? Memory extracted: Name='Bob', Persona='data scientist'")

time.sleep(1)

# Follow-up question
message_with_memory_2 = {
    "message": "What's my name and what do I do?",
    "session_id": session_with_memory,
    "agents": ["azure_openai_agent"],
    "enable_memory": True
}

print(f"\n[Turn 2] ?? User: {message_with_memory_2['message']}")
print(f"         Memory: {'ON ??' if message_with_memory_2['enable_memory'] else 'OFF ??'}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=message_with_memory_2,
    timeout=TIMEOUT
)
result_with_mem_2 = response.json()

print(f"?? Agent: {result_with_mem_2['content']}")
print(f"\n? Success! Agent remembered your name and role!")



?? Scenario 2: Memory ENABLED

[Turn 1] ?? User: Hi! My name is Bob and I'm a data scientist.
         Memory: ON ??

?? Agent: Hi Bob! It’s great to meet you. As a data scientist, you probably tackle a variety of interesting problems. Is there a specific project or topic you’re working on right now that I can help with?

?? Session ID: d474e5bb-1b63-44ed-8c29-98369e21c7e8
? Memory extracted: Name='Bob', Persona='data scientist'

[Turn 2] ?? User: What's my name and what do I do?
         Memory: ON ??

?? Agent: Your name is Bob, and you’re a data scientist.

? Success! Agent remembered your name and role!


## Scenario 3: Multi-Turn Personalized Conversation

See how memory enables personalized, context-aware responses across multiple turns.

In [6]:
print("\n\n?? Scenario 3: Personalized Multi-Turn Conversation")
print("="*50 + "\n")

# Start new session with detailed introduction
personalized_msg_1 = {
    "message": "Hello! I'm Sarah, a marketing manager at TechCorp, and I'm 28 years old.",
    "agents": ["azure_openai_agent"],
    "enable_memory": True
}

print(f"[Turn 1] ?? User: {personalized_msg_1['message']}")
print(f"         Memory: ON ??\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=personalized_msg_1,
    timeout=TIMEOUT
)
personal_result_1 = response.json()
personal_session = personal_result_1["session_id"]

print(f"?? Agent: {personal_result_1['content']}")
print(f"\n? Extracted: Name='Sarah', Persona='marketing manager at TechCorp, 28 years old'")

# Series of follow-up questions
follow_up_questions = [
    "Can you remind me what I do for work?",
    "What's my name again?",
    "Summarize what you know about me.",
    "Give me some career advice based on my background."
]

turn_num = 2
for question in follow_up_questions:
    time.sleep(1)
    
    print(f"\n[Turn {turn_num}] ?? User: {question}")
    print(f"           Memory: ON ??\n")
    
    follow_up_msg = {
        "message": question,
        "session_id": personal_session,
        "agents": ["azure_openai_agent"],
        "enable_memory": True
    }
    
    response = session.post(
        f"{API_BASE_URL}/chat",
        json=follow_up_msg,
        timeout=TIMEOUT
    )
    follow_up_result = response.json()
    
    print(f"?? Agent: {follow_up_result['content']}")
    
    turn_num += 1

print(f"\n\n? All responses were personalized using remembered information!")



?? Scenario 3: Personalized Multi-Turn Conversation

[Turn 1] ?? User: Hello! I'm Sarah, a marketing manager at TechCorp, and I'm 28 years old.
         Memory: ON ??

?? Agent: Hello Sarah! It’s great to meet you. As a marketing manager at TechCorp, you must have a lot on your plate. How can I assist you today? Are you looking for help with marketing strategies, campaign ideas, analytics, or something else?

? Extracted: Name='Sarah', Persona='marketing manager at TechCorp, 28 years old'

[Turn 2] ?? User: Can you remind me what I do for work?
           Memory: ON ??

?? Agent: Of course! You mentioned that you’re a marketing manager at TechCorp. Your role likely involves planning and executing marketing campaigns, analyzing market trends, managing social media and content, collaborating with teams, and working to increase brand awareness and engagement. If you’d like, I can help with specific tasks or provide resources related to marketing management!

[Turn 3] ?? User: What's my 

## Scenario 4: Memory with Multiple Agents (Group Chat)

UserInfoMemory works seamlessly in multi-agent scenarios.

In [9]:
print("\n\n?? Scenario 4: Memory with Multiple Agents")
print("="*50 + "\n")

# First message with multiple agents
multi_agent_mem_msg_1 = {
    "message": "Hi! I'm Michael, a cloud architect specializing in Azure. I need help planning a migration.",
    "agents": ["azure_openai_agent", "ms_foundry_people_agent"],
    "max_turns": 3,
    "enable_memory": True,
    "format": "user_friendly"
}

print(f"[Turn 1] ?? User: {multi_agent_mem_msg_1['message']}")
print(f"         Memory: ON ??")
print(f"         Agents: {', '.join(multi_agent_mem_msg_1['agents'])}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=multi_agent_mem_msg_1,
    timeout=TIMEOUT
)
response.raise_for_status()
multi_mem_result_1 = response.json()
multi_mem_session = multi_mem_result_1.get("session_id")
if not multi_mem_session:
    raise ValueError(f"Session ID missing in response: {multi_mem_result_1}")

content_1 = multi_mem_result_1.get("content", "")
truncated_content = content_1[:300] + "..." if len(content_1) > 300 else content_1
print(f"?? Response: {truncated_content}")
print(f"\n? Memory extracted: Name='Michael', Persona='cloud architect specializing in Azure'")

time.sleep(1.5)

# Follow-up leveraging memory
multi_agent_mem_msg_2 = {
    "message": "What expertise do I have that would help with this migration?",
    "session_id": multi_mem_session,
    "agents": ["azure_openai_agent", "ms_foundry_people_agent"],
    "max_turns": 3,
    "enable_memory": True,
    "format": "user_friendly"
}

print(f"\n[Turn 2] ?? User: {multi_agent_mem_msg_2['message']}")
print(f"         Memory: ON ??\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=multi_agent_mem_msg_2,
    timeout=TIMEOUT
)
response.raise_for_status()
multi_mem_result_2 = response.json()

content_2 = multi_mem_result_2.get("content", "")
truncated_content_2 = content_2[:300] + "..." if len(content_2) > 300 else content_2
print(f"?? Response: {truncated_content_2}")

# Show metadata
if "metadata" in multi_mem_result_2 and multi_mem_result_2["metadata"]:
    metadata = multi_mem_result_2["metadata"]
    print(f"\n?? Collaboration Stats:")
    print(f"   Agent Count: {metadata.get('agent_count', 'N/A')}")
    
    if "contributing_agents" in metadata:
        agent_list = ", ".join(metadata["contributing_agents"])
        print(f"   Agents Used: {agent_list}")

print(f"\n? All agents in the group chat have access to the same memory!")



?? Scenario 4: Memory with Multiple Agents

[Turn 1] ?? User: Hi! I'm Michael, a cloud architect specializing in Azure. I need help planning a migration.
         Memory: ON ??
         Agents: azure_openai_agent, ms_foundry_people_agent



HTTPError: 500 Server Error: Internal Server Error for url: http://localhost:8000/chat

## Scenario 5: Testing Information Extraction Patterns

See how UserInfoMemory extracts different types of information.

In [10]:
print("\n\n?? Scenario 5: Information Extraction Patterns")
print("="*50 + "\n")

extraction_tests = [
    {"Input": "My name is Jennifer", "Expected": "Name: Jennifer"},
    {"Input": "I'm a developer", "Expected": "Persona: developer"},
    {"Input": "Call me Mike and I work as a project manager", "Expected": "Name: Mike, Persona: project manager"},
    {"Input": "I'm 35 and I'm a teacher", "Expected": "Persona: 35 years old, teacher"}
]

test_num = 1
for test in extraction_tests:
    print(f"\n[Test {test_num}]")
    print(f"Input: \"{test['Input']}\"")
    print(f"Expected: {test['Expected']}\n")
    
    extract_msg = {
        "message": test["Input"],
        "agents": ["azure_openai_agent"],
        "enable_memory": True
    }
    
    response = session.post(
        f"{API_BASE_URL}/chat",
        json=extract_msg,
        timeout=TIMEOUT
    )
    extract_result = response.json()
    
    print(f"?? Agent Response: {extract_result['content']}")
    print(f"? Information extracted and stored in session")
    
    test_num += 1
    time.sleep(0.8)

print(f"\n\n? UserInfoMemory successfully extracts various information patterns!")



?? Scenario 5: Information Extraction Patterns


[Test 1]
Input: "My name is Jennifer"
Expected: Name: Jennifer

?? Agent Response: Hello, Jennifer! Nice to meet you. How can I assist you today?
? Information extracted and stored in session

[Test 2]
Input: "I'm a developer"
Expected: Persona: developer

?? Agent Response: Great! Welcome. How can I assist you today? Are you working on a specific project, looking for coding help, or interested in discussing technology trends? Let me know what you need!
? Information extracted and stored in session

[Test 3]
Input: "Call me Mike and I work as a project manager"
Expected: Name: Mike, Persona: project manager

?? Agent Response: Great to meet you, Mike! As a project manager, you probably juggle a lot of responsibilities. If you need help with project planning, tracking tasks, managing teams, or anything else related to your role, just let me know how I can assist you.
? Information extracted and stored in session

[Test 4]
Input: "I'm 35

## Best Practices for Using Memory

In [11]:
print("?? UserInfoMemory Best Practices")
print("="*50 + "\n")

print("? DO:")
print("   1. Enable memory for personalized, multi-turn conversations")
print("   2. Let users introduce themselves naturally")
print("   3. Use memory in customer service scenarios")
print("   4. Combine memory with session management for best results")
print("   5. Provide a UI toggle to let users control memory")
print()

print("? DON'T:")
print("   1. Enable memory for anonymous/privacy-sensitive conversations")
print("   2. Store sensitive personal information via memory")
print("   3. Expect memory to work without a session_id")
print("   4. Use memory for one-off queries")
print()

print("?? Tips:")
print("   � Memory extracts: name, persona/role, age")
print("   � Works best with explicit information (\"My name is...\", \"I'm a...\")")
print("   � Memory persists for the entire session")
print("   � Can be enabled per-request with 'enable_memory' flag")
print("   � Falls back gracefully if extraction fails")
print()

print("?? When to Use Memory:")
print("   ? Customer support conversations")
print("   ? Personal assistant scenarios")
print("   ? Tutoring or coaching interactions")
print("   ? Long-running task assistance")
print()

print("?? When NOT to Use Memory:")
print("   ? Anonymous information queries")
print("   ? Privacy-sensitive conversations")
print("   ? Quick one-time questions")
print("   ? Public/shared agent instances")

?? UserInfoMemory Best Practices

? DO:
   1. Enable memory for personalized, multi-turn conversations
   2. Let users introduce themselves naturally
   3. Use memory in customer service scenarios
   4. Combine memory with session management for best results
   5. Provide a UI toggle to let users control memory

? DON'T:
   1. Enable memory for anonymous/privacy-sensitive conversations
   2. Store sensitive personal information via memory
   3. Expect memory to work without a session_id
   4. Use memory for one-off queries

?? Tips:
   � Memory extracts: name, persona/role, age
   � Works best with explicit information ("My name is...", "I'm a...")
   � Memory persists for the entire session
   � Can be enabled per-request with 'enable_memory' flag
   � Falls back gracefully if extraction fails

?? When to Use Memory:
   ? Customer support conversations
   ? Personal assistant scenarios
   ? Tutoring or coaching interactions
   ? Long-running task assistance

?? When NOT to Use Memory:

## Memory Configuration and Control

In [12]:
print("?? Memory Configuration")
print("="*50 + "\n")

print("??? Control Methods:")
print()

print("1. Request-level (Recommended):")
print("   POST /chat")
print("   {")
print("     \"message\": \"Your message\",")
print("     \"enable_memory\": true    ? Control per request")
print("   }")
print()

print("2. Environment Variable (Default):")
print("   File: .env")
print("   ENABLE_LONG_RUNNING_MEMORY=\"true\"")
print()

print("3. UI Toggle (Frontend):")
print("   � React: <MemoryToggle> component")
print("   � HTML: memory-demo.html")
print("   � Main App: Sidebar toggle in App.js")
print()

print("?? Priority Order:")
print("   1. Request Flag (enable_memory) - Highest priority")
print("   2. Environment Variable (ENABLE_LONG_RUNNING_MEMORY)")
print("   3. Default (false) - Lowest priority")
print()

print("?? Memory State:")
print("   � Stored in: Session state (UserInfoMemory provider)")
print("   � Lifetime: Duration of session")
print("   � Scope: Per-session (isolated between sessions)")
print("   � Persistence: In-memory (not persisted to disk)")
print()

print("?? What Gets Stored:")
print("   � UserName: Extracted from name patterns")
print("   � UserPersona: Role, occupation, age")
print("   � HasAskedForName: Internal tracking flag")
print("   � HasAskedForPersona: Internal tracking flag")

?? Memory Configuration

??? Control Methods:

1. Request-level (Recommended):
   POST /chat
   {
     "message": "Your message",
     "enable_memory": true    ? Control per request
   }

2. Environment Variable (Default):
   File: .env
   ENABLE_LONG_RUNNING_MEMORY="true"

3. UI Toggle (Frontend):
   � React: <MemoryToggle> component
   � HTML: memory-demo.html
   � Main App: Sidebar toggle in App.js

?? Priority Order:
   1. Request Flag (enable_memory) - Highest priority
   2. Environment Variable (ENABLE_LONG_RUNNING_MEMORY)
   3. Default (false) - Lowest priority

?? Memory State:
   � Stored in: Session state (UserInfoMemory provider)
   � Lifetime: Duration of session
   � Scope: Per-session (isolated between sessions)
   � Persistence: In-memory (not persisted to disk)

?? What Gets Stored:
   � UserName: Extracted from name patterns
   � UserPersona: Role, occupation, age
   � HasAskedForName: Internal tracking flag
   � HasAskedForPersona: Internal tracking flag


## Summary

In this notebook, you learned:
- ? How to enable/disable UserInfoMemory with `enable_memory` flag
- ? How memory extracts name and persona from user messages
- ? Difference between memory-enabled and disabled conversations
- ? How memory enhances personalization across multiple turns
- ? Memory works seamlessly with multi-agent scenarios
- ? Various information extraction patterns
- ? Best practices and when to use memory

## Key Features

### UserInfoMemory
- Automatically extracts user name and persona from messages
- Stores information in session state
- Personalizes agent responses using remembered info
- Works with both single and multi-agent conversations

### Control & Configuration
- Per-request control via `enable_memory` flag
- Environment variable for default behavior
- UI toggles in frontend applications
- Priority: Request > Environment > Default

### Extraction Patterns
- **Name**: "My name is X", "I'm X", "Call me X"
- **Persona**: "I'm a developer", "I work as X", "I'm 25"
- Falls back gracefully if extraction fails

## Real-World Use Cases
1. **Customer Support**: Remember customer details across conversation
2. **Personal Assistants**: Personalize suggestions based on user role
3. **Coaching/Tutoring**: Adapt teaching style to student background
4. **Sales**: Remember customer preferences and needs

## UI Integration
- **Main App**: Memory toggle in sidebar (App.js)
- **HTML Demo**: memory-demo.html with live stats
- **React Component**: MemoryToggle.jsx (reusable)
- **Debug Tool**: memory-test.html for testing

## Next Steps
- Try the frontend demos to see memory in action
- Review the **Single Agent Demo** for basic agent interaction
- Explore the **Multiple Agents Demo** for group chat scenarios
- Check the **Content Safety Demo** for content moderation